<a href="https://colab.research.google.com/github/adityasharma10699/demopygit/blob/main/Completionrates_LR%2BRFR%2BSVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
import plotly.graph_objs as go
import plotly.figure_factory as ff

# Importing dataset and examining it
dataset = pd.read_csv("/content/drive/MyDrive/AS & ML/Regression/CompletionRates.csv")
pd.set_option('display.max_columns', None) # to make sure you can see all the columns in output window
print(dataset.head())
print(dataset.shape)
print(dataset.info())
print(dataset.describe())

#Dummies
categorical_features = ['INST_TYPE']
final_data = pd.get_dummies(dataset, columns = categorical_features, drop_first = True)
print(final_data.info())
print(final_data.head(2))


# Plotting Correlation Heatmap
corrs = final_data.corr()
figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True)
figure.show()

# Dividing dataset into label and feature sets
X = final_data.drop(['HECR','TS_MEN','ST_FI_LO','ST_FI_M1','ST_FI_M2','ST_FI_H1','ST_FI_H2','HL_ED_P_PS','TS_DPEN','AVG_FAC_SAL_PM'], axis = 1) # Features
Y = final_data['HECR'] # Labels
print(type(X))
print(type(Y))
print(X.shape)
print(Y.shape)

# Normalizing numerical features so that each feature has mean 0 and variance 1
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

# Linear Regression with Regularization
# Tuning the SGDRegressor parameters 'eta0' (learning rate) and 'max_iter', along with the regularization parameter alpha using Grid Search
sgdr = SGDRegressor(random_state = 1, penalty = 'elasticnet')
grid_param = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[10000, 20000, 30000, 40000],'alpha': [.001, .01, .1, 1,10, 100], 'l1_ratio': [0,0.25,0.5,0.75,1]}

gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, Y)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("r2: ", best_result)

Adj_r2 = 1-(1-best_result)*(11440-1)/(11440-11-1)
print("Adjusted r2: ", Adj_r2)

# '''
# Adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

# where, n = number of observations in training data, p = number of features
# '''

best_model = gd_sr.best_estimator_
print("Intercept: ", best_model.intercept_)

print(pd.DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

##################################################################################
# Implementing Random Forest Regression
# Tuning the random forest parameter 'n_estimators' and implementing cross-validation using Grid Search
rfr = RandomForestRegressor(criterion='squared_error', max_features='sqrt', random_state=1)
grid_param = {'n_estimators': [10,20,30,40,50,100]}

gd_sr = GridSearchCV(estimator=rfr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, Y)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("r2: ", best_result)

Adj_r2 = 1-(1-best_result)*(11440-1)/(11440-11-1)
print("Adjusted r2: ", Adj_r2)

# '''
# Adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

# where, n = number of observations in training data, p = number of features
# '''

featimp = pd.Series(gd_sr.best_estimator_.feature_importances_, index=list(X)).sort_values(ascending=False) # Getting feature importances list for the best model
print(featimp)

# # Selecting features with higher sifnificance and redefining feature set
# X_ = dataset[['GrLivArea', 'YearBuilt', 'TotalBsmtSF', 'GarageArea']]

# feature_scaler = StandardScaler()
# X_scaled_ = feature_scaler.fit_transform(X_)

# # Tuning the random forest parameter 'n_estimators' and implementing cross-validation using Grid Search
# rfr = RandomForestRegressor(criterion='squared_error', max_features='sqrt', random_state=1)
# grid_param = {'n_estimators': [50,100,150,200,250]}

# gd_sr = GridSearchCV(estimator=rfr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled_, Y)

# best_parameters = gd_sr.best_params_
# print("Best parameters: ", best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print("r2: ", best_result)

# Adj_r2 = 1-(1-best_result)*(11440-1)/(11440-11-1)
# print("Adjusted r2: ", Adj_r2)

# '''
# Adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

# where, n = number of observations in training data, p = number of features
# '''
####################################################################################
# Implementing Support Vector Regression
# Tuning the SVR parameters 'kernel', 'C', 'epsilon' and implementing cross-validation using Grid Search
svr = SVR()
grid_param = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [.001,.01, 0.1, 1, 10,100]}

gd_sr = GridSearchCV(estimator=svr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, Y)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("r2: ", best_result)

Adj_r2 = 1-(1-best_result)*(11440-1)/(11440-11-1)
print("Adjusted r2: ", Adj_r2)

   HIGH_DEG  INST_TYPE  INST_EXP_PS  AVG_FAC_SAL_PM     TS  TS_MEN  TS_WOM  \
0         4          1         3212            4587   3903  0.4766  0.5234   
1         4          1        10418            6398   9562  0.4195  0.5805   
2         4          1         5784            5859   4165  0.4898  0.5102   
3         4          1         5824            6109  14245  0.4795  0.5205   
4         4          1         4622            6102  18520  0.5212  0.4788   

     TS_MAR   TS_DPEN    TS_VET  PER_PT  ST_FI_LO  ST_FI_M1  ST_FI_M2  \
0  0.056698  0.821877  0.018063  0.0699  0.504265  0.172604  0.136979   
1  0.149349  0.590745  0.039623  0.3095  0.490163  0.155722  0.144638   
2  0.221490  0.551088  0.048780  0.3160  0.440343  0.156229  0.154252   
3  0.065476  0.815247  0.024038  0.1046  0.331731  0.133242  0.156364   
4  0.061376  0.825645  0.025997  0.0821  0.272478  0.126271  0.150899   

   ST_FI_H1  ST_FI_H2  HL_ED_P_MS  HL_ED_P_HS  HL_ED_P_PS    HECR  
0  0.110386  0.075765   

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
(14300, 11)
(14300,)
Best parameters:  {'alpha': 0.001, 'eta0': 0.001, 'l1_ratio': 0, 'max_iter': 10000}
r2:  0.45405505411428493
Adjusted r2:  0.4535295558289557
Intercept:  [0.50959651]
       Features  Coefficients
9   INST_TYPE_2      0.042238
2            TS      0.037360
10  INST_TYPE_3      0.036259
1   INST_EXP_PS      0.034467
3        TS_WOM      0.010960
5        TS_VET      0.002977
0      HIGH_DEG     -0.002107
7    HL_ED_P_MS     -0.006638
4        TS_MAR     -0.006990
6        PER_PT     -0.029927
8    HL_ED_P_HS     -0.042174
Best parameters:  {'n_estimators': 100}
r2:  0.6472127421292171
Adjusted r2:  0.6468731674147807
HL_ED_P_HS     0.213550
PER_PT         0.166551
INST_EXP_PS    0.159850
TS             0.103138
TS_MAR         0.096876
HL_ED_P_MS     0.074457
TS_WOM         0.071591
TS_VET         0.060693
INST_TYPE_2    0.032690
HIGH_DEG       0.015469
INST_TYPE_3    0.005134
dtype: float64
Be

In [ ]:
from google.colab import drive
drive.mount('/content/drive')